# Weather Dataset Cleaning
In this notebook we will clean the weather dataset from the KNMI

### First check and arranges
First we will import some modules we may need.

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
from collections import Counter

We can read the excel file directly. We inspect it.

In [2]:
weather = pd.read_excel('KNMI_weather_data_cleaning.xlsx')

In [3]:
#weather.head()
print(weather.dtypes)
print(weather.shape)

STN           int64
YYYYMMDD      int64
HH            int64
T             int64
T10         float64
TD            int64
SQ            int64
DR            int64
RH            int64
P             int64
U             int64
WW          float64
IX            int64
R           float64
S           float64
O           float64
dtype: object
(17520, 16)


* We've seen that some fields, like temperature and sunshine have to be divided by 10. They come in 0.1 hours or 0.1 mm for example. As we are working with hourly data we will feel more comfortable converting them to hours or mm/h.

In [4]:
divisibles = ['T', 'T10', 'TD', 'P', 'SQ', 'DR', 'RH']

for i in divisibles:
    weather[i] = weather[i]/10

* We will also drop some unnecessary columns:
 * STN because it's the station, and it's the same for all the registers.
 * T10 it's minimum temperature at 10cm.

In [5]:
weather = weather.drop(columns=['STN', 'T10', 'TD'])

* We see it's a bit complicated to work with these names abreviations so we will rename the columns.

In [6]:
weather = weather.rename(columns={"YYYYMMDD": "Date", "HH": "Hour", "T": "Temperature", 
                                  "SQ": "Sun", "DR": "PpDur", "RH": "PpAmount",
                                  "P": "Pressure", "U": "Humidity", "R": "Rain", "S": "Snow", 
                                  "O": "Thunderstorm"})

* In precipitation amount values, -1 was given if precipitation was <0.05mm, we will just change that to 0, as it could bring some problems later

In [7]:
weather['PpAmount'] = weather['PpAmount'].apply(lambda x: 0 if x == -0.1 else x)
weather.head()

,Date,Hour,Temperature,Sun,PpDur,PpAmount,Pressure,Humidity,WW,IX,Rain,Snow,Thunderstorm
0,20130101,1,8.7,0.0,1.0,1.7,998.3,93,52.0,7,1.0,0.0,0.0
1,20130101,2,8.6,0.0,1.0,1.8,998.6,94,58.0,7,1.0,0.0,0.0
2,20130101,3,8.6,0.0,0.7,1.7,998.4,96,58.0,7,1.0,0.0,0.0
3,20130101,4,9.1,0.0,0.0,0.0,998.3,94,23.0,7,1.0,0.0,0.0
4,20130101,5,8.9,0.0,0.0,0.0,998.4,95,NaN,5,0.0,0.0,0.0


* We will do another step that may help us later to give information about the rain. We will create a new column and we will classify the rain intensity in categories. We do it now because rain doesn't last all day and we can use mm/h to do it.
    * Slight: < 2.5 mm/h
    * Moderate: > 2.5 mm/h and < 7.5 mm/h
    * Heavy: > 7.5 mm/h and < 20 mm/h
    * Very Heavy: > 20 mm/h and < 50 mm/h
    * Torrential: > 50 mm/h

In [8]:
weather.insert(6, 'PpLabel', value = '', allow_duplicates = True) 

for i,row in weather.iterrows():
    if weather.loc[i, 'PpAmount'] == 0:
        weather.loc[i, 'PpLabel'] = 'None'
    elif weather.loc[i, 'PpAmount'] > 0 and weather.loc[i, 'PpAmount'] <= 2.5:
        weather.loc[i, 'PpLabel'] = 'Slight'
    elif weather.loc[i, 'PpAmount'] > 2.5 and weather.loc[i, 'PpAmount'] <= 7.5:
        weather.loc[i, 'PpLabel'] = 'Moderate'
    elif weather.loc[i, 'PpAmount'] > 7.5 and weather.loc[i, 'PpAmount'] <= 20:
        weather.loc[i, 'PpLabel'] = 'Heavy'
    elif weather.loc[i, 'PpAmount'] > 20 and weather.loc[i, 'PpAmount'] <= 50:
        weather.loc[i, 'PpLabel'] = 'Very Heavy'   
    elif weather.loc[i, 'PpAmount'] >= 50:
        weather.loc[i, 'PpLabel'] = 'Torrential'

weather.head(24)

,Date,Hour,Temperature,Sun,PpDur,PpAmount,PpLabel,Pressure,Humidity,WW,IX,Rain,Snow,Thunderstorm
0,20130101,1,8.7,0.0,1.0,1.7,Slight,998.3,93,52.0,7,1.0,0.0,0.0
1,20130101,2,8.6,0.0,1.0,1.8,Slight,998.6,94,58.0,7,1.0,0.0,0.0
2,20130101,3,8.6,0.0,0.7,1.7,Slight,998.4,96,58.0,7,1.0,0.0,0.0
3,20130101,4,9.1,0.0,0.0,0.0,None,998.3,94,23.0,7,1.0,0.0,0.0
4,20130101,5,8.9,0.0,0.0,0.0,None,998.4,95,NaN,5,0.0,0.0,0.0
5,20130101,6,8.1,0.0,0.5,0.2,Slight,998.7,94,57.0,7,1.0,0.0,0.0
6,20130101,7,7.9,0.0,0.3,0.1,Slight,999.4,94,23.0,7,1.0,0.0,0.0
7,20130101,8,7.7,0.0,0.6,0.3,Slight,1000.1,96,57.0,7,1.0,0.0,0.0
8,20130101,9,6.7,0.0,0.9,0.8,Slight,1001.4,93,62.0,7,1.0,0.0,0.0
9,20130101,10,6.8,0.0,0.7,1.2,Slight,1002.4,91,23.0,7,1.0,0.0,0.0


### Creating a daily dataset
Because the tweets dataset is in daily format and we want to compare the weather within these days, we will transform the weather hourly dataset to daily format.
* Here we are going to create a new dataset, where we will store the daily calculations and values instead of hourly.

In [9]:
columnsweather = ['Day', 'MinTemp', 'MaxTemp', 'AvgTemp', 'TempVar', 'SunTime', 'PpDur', 'PpAmount', 'MinPres', 
                  'MaxPres', 'AvgPres', 'MinHum', 'MaxHum', 'AvgHum', 'Rain', 'Snow', 'TStorm']
dailyweather = pd.DataFrame(columns = columnsweather)
dailyweather.head()

,Day,MinTemp,MaxTemp,AvgTemp,TempVar,SunTime,PpDur,PpAmount,MinPres,MaxPres,AvgPres,MinHum,MaxHum,AvgHum,Rain,Snow,TStorm


* First we will copy the date column of the hourly dataset, and we will modify that in order that each day appears only once.

In [10]:
dailyweather['Day'] = weather['Date']
dailyweather.drop_duplicates(inplace = True)
dailyweather.reset_index(drop=True, inplace=True)

* Now we will do the calculations and fill the values for each row/column
    * For temperatures: minimum, maximum, average and daily temperature variation.
    * For sun time: total suntime
    * For precipitation: total daily precipitation time and total daily precipitation amount.
    * For pressure: minumum, maximum and average.
    * For humidity: minimum, maximum and average.
    * For rain: 'Yes' if it rained for more than 3 hours.
    * For snow: 'Yes' if it rained for more than 3 hours.
    * For thunderstorm: 'Yes' if it stormed for more than 2 hours.

In [11]:
y = 0
z = 24
for i, row in dailyweather.iterrows():
    # Regarding temperature
    dailyweather.loc[i, 'MinTemp'] = min(weather[y:z]['Temperature'])
    dailyweather.loc[i, 'MaxTemp'] = max(weather[y:z]['Temperature'])
    dailyweather.loc[i, 'AvgTemp'] = weather[y:z]['Temperature'].mean()
    dailyweather.loc[i, 'TempVar'] = max(weather[y:z]['Temperature']) - min(weather[y:z]['Temperature'])
    # Suntime
    dailyweather.loc[i, 'SunTime'] = sum(weather[y:z]['Sun'])
    # Precipitation
    dailyweather.loc[i, 'PpDur'] = sum(weather[y:z]['PpDur'])
    dailyweather.loc[i, 'PpAmount'] = sum(weather[y:z]['PpAmount'])
    # Pressure
    dailyweather.loc[i, 'MinPres'] = min(weather[y:z]['Pressure'])
    dailyweather.loc[i, 'MaxPres'] = max(weather[y:z]['Pressure'])
    dailyweather.loc[i, 'AvgPres'] = weather[y:z]['Pressure'].mean()
    # Humidity
    dailyweather.loc[i, 'MinHum'] = min(weather[y:z]['Humidity'])
    dailyweather.loc[i, 'MaxHum'] = max(weather[y:z]['Humidity'])
    dailyweather.loc[i, 'AvgHum'] = weather[y:z]['Humidity'].mean()
    # Rain, Snow and Thunderstorm 'Yes' or 'No'
    if sum(weather[y:z]['Rain']) > 3:
        dailyweather.loc[i, 'Rain'] = 'Yes'
    else: 
        dailyweather.loc[i, 'Rain'] = 'No'
    
    if sum(weather[y:z]['Snow']) > 2:
        dailyweather.loc[i, 'Snow'] = 'Yes'
    else: 
        dailyweather.loc[i, 'Snow'] = 'No'
        
    if sum(weather[y:z]['Thunderstorm']) > 2:
        dailyweather.loc[i, 'TStorm'] = 'Yes'
    else: 
        dailyweather.loc[i, 'TStorm'] = 'No'

    y = y + 24
    z = z + 24
    
dailyweather.head(15)
dailyweather.to_csv('dailyweather.csv')

#Below some other informative options we may need

#pd.set_option('display.max_rows', dailyweather.shape[0]+1)
#print(dailyweather.describe(include='all').transpose())
#pandas_profiling.ProfileReport(dailyweather)
#print(dailyweather)

In [12]:
dailyweather['Day'] = dailyweather.Day.astype(str)
dailyweather['Day'].dtypes
dailyweather['Day'].head(3)

0    20130101
1    20130102
2    20130103
Name: Day, dtype: object

In [13]:
dailyweather.head(10)

,Day,MinTemp,MaxTemp,AvgTemp,TempVar,SunTime,PpDur,PpAmount,MinPres,MaxPres,AvgPres,MinHum,MaxHum,AvgHum,Rain,Snow,TStorm
0,20130101,5.7,9.1,7.14167,3.4,2.6,6.4,8.1,998.3,1012.4,1004.42,76,96,86.0833,Yes,No,No
1,20130102,4.2,8.2,6.80417,4,3.4,1.6,0.4,1013.4,1024.2,1021.11,82,98,88.5417,No,No,No
2,20130103,8.8,11.2,10.1125,2.4,2.5,0.5,0.1,1023.7,1034.8,1029.68,89,98,92.75,No,No,No
3,20130104,8.4,9.9,9.10833,1.5,0,0,0,1033.6,1035.4,1034.54,88,97,93.4167,No,No,No
4,20130105,7.4,8.8,8.14167,1.4,0,1.9,0.3,1031.5,1034.6,1033.45,88,97,93.625,Yes,No,No
5,20130106,5.2,10.6,7.99167,5.4,4,0,0,1029.7,1031.2,1030.36,87,100,94.7917,No,No,No
6,20130107,6.1,8.2,7.14167,2.1,0,0,0,1025.1,1029.7,1027.16,89,98,95.7083,Yes,No,No
7,20130108,6.2,9.1,7.69583,2.9,0,1.8,0.3,1022.2,1024.9,1023.86,88,98,93.4583,Yes,No,No
8,20130109,2.2,8.3,6.65833,6.1,0,11.8,6.2,1016.4,1021.8,1018.96,85,98,93.9167,Yes,No,No
9,20130110,0,7.8,5.43333,7.8,3.1,2.1,2.9,1011.3,1016.1,1013.24,78,95,85.8333,Yes,No,No


### Daily descriptive tags / labels 
* In this part we will create a new dataframe containing labels representing the characteristics of each day. So we will be able to easily categorize the days without looking at numbers (however, we will still have and use the other dataset, if needed)

In [13]:
columnslabels = ['Day', 'Temperature', 'TempVariation', 'Sunny', 'Pressure', 'Humidity', 'Rainy', 'Snow', 'Thunderstorm']
weatherlabels = pd.DataFrame(columns=columnslabels)

weatherlabels['Day'] = dailyweather['Day']
weatherlabels.head()

,Day,Temperature,TempVariation,Sunny,Pressure,Humidity,Rainy,Snow,Thunderstorm
0,20130101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20130102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20130103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20130104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20130105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* For labeling, we used some criteria:
    * **Temperature** (based on the daily average)
        * **Freezing**: < 0ºC.
        * **Cold**: > 0º and < 10º.
        * **Mild**: > 10º and < 16º.
        * **Warm**: > 16º and < 27º.
        * **Hot**: > 27ºC. 
    * **Pressure** (based on the daily average)
        * **Low**: Less than 1013.25 hPa.
        * **High**: More than 1013.25 hPa.
    * **Humidity** (based on the daily average)
        * **Unconfortable - High**: For less than 20º, more than 85%. Between 20º and 26º, more than 72.5%. More than 26º, more than 60%.
        * **Comfortable**: For less than 20º, between 30% and 85%. Between 20º and 26º, between 25% and 72.5%. More than 26º, Between 20% and 60%.
        * **Unconfortable - Low**: For less than 20º, less than 30%. Between 20º and 26º, less than 25%. More than 26º, less than 20%.
        We based this on a simplification of [this table](https://www.researchgate.net/profile/Ali_Alahmer/publication/259359809/figure/fig1/AS:390976431509505@1470227453531/RELATIVE-HUMIDITY-RH-TEMPERATURE-T-DIAGRAM-BASED-ON-COMFORT-ZONE-ACCORDING-TO.png).
    * **Snow and thunderstorm**
        * Simply **'Yes'** or **'No'** if there's been some in that day.

In [14]:
y = 0
x = 7
o = 20
z = 24

for i, row in weatherlabels.iterrows():
    # Classifying temperatures
    if dailyweather['AvgTemp'][i] <= 0:
        weatherlabels['Temperature'][i] = 'Freezing'
    elif dailyweather['AvgTemp'][i] < 10:
        weatherlabels['Temperature'][i] = 'Cold'
    elif dailyweather['AvgTemp'][i] < 16:
        weatherlabels['Temperature'][i] = 'Mild'
    elif dailyweather['AvgTemp'][i] < 27:
        weatherlabels['Temperature'][i] = 'Warm'
    elif dailyweather['AvgTemp'][i] >= 27:
        weatherlabels['Temperature'][i] = 'Hot'
    # Classifying temperature variation
    if dailyweather['TempVar'][i] >= 12:
        weatherlabels['TempVariation'][i] = 'High'
    elif dailyweather['TempVar'][i] > 4 and dailyweather['TempVar'][i] < 12:
        weatherlabels['TempVariation'][i] = 'Moderate'
    elif dailyweather['TempVar'][i] <= 4:
        weatherlabels['TempVariation'][i] = 'Low'
    # Classifying sun time
    if dailyweather['SunTime'][i] >= 10:
        weatherlabels['Sunny'][i] = 'Very Sunny'
    elif dailyweather['SunTime'][i] > 5 and dailyweather['TempVar'][i] < 10:
        weatherlabels['Sunny'][i] = 'Sunny'
    elif dailyweather['SunTime'][i] <= 5:
        weatherlabels['Sunny'][i] = 'Not Sunny'
    
    # Classifying pressure
    if dailyweather['AvgPres'][i] <= 1013.25:
        weatherlabels['Pressure'][i] = 'Low'
    else:
        weatherlabels['Pressure'][i] = 'High'
    # Classifying himidity levels (google "relative humidity comfort")
    # Block temperature less than 20º
    if dailyweather['AvgTemp'][i] <= 20 and dailyweather['AvgHum'][i] >= 85:
        weatherlabels['Humidity'][i] = 'Uncomfortable - High'
    elif dailyweather['AvgTemp'][i] <= 20 and dailyweather['AvgHum'][i] < 85 and dailyweather['AvgHum'][i] > 30:
        weatherlabels['Humidity'][i] = 'Comfortable'        
    elif dailyweather['AvgTemp'][i] <= 20 and dailyweather['AvgHum'][i] <= 30:
        weatherlabels['Humidity'][i] = 'Uncomfortable - Low'
    # Block temperature between 20º and 26º 
    if dailyweather['AvgTemp'][i] > 20 and dailyweather['AvgTemp'][i] < 26 and dailyweather['AvgHum'][i] >= 72.5:
        weatherlabels['Humidity'][i] = 'Uncomfortable - High'
    elif dailyweather['AvgTemp'][i] > 20 and dailyweather['AvgTemp'][i] < 26 and dailyweather['AvgHum'][i] < 72.5 and dailyweather['AvgHum'][i] > 25:
        weatherlabels['Humidity'][i] = 'Comfortable'        
    elif dailyweather['AvgTemp'][i] > 20 and dailyweather['AvgTemp'][i] < 26 and dailyweather['AvgHum'][i] <= 25:
        weatherlabels['Humidity'][i] = 'Uncomfortable - Low'
    # Block temperature more than 26
    if dailyweather['AvgTemp'][i] >= 26 and dailyweather['AvgHum'][i] >= 60:
        weatherlabels['Humidity'][i] = 'Uncomfortable - High'
    elif dailyweather['AvgTemp'][i] >= 26 and dailyweather['AvgHum'][i] < 60 and dailyweather['AvgHum'][i] > 20:
        weatherlabels['Humidity'][i] = 'Comfortable'        
    elif dailyweather['AvgTemp'][i] >= 26 and dailyweather['AvgHum'][i] <= 20:
        weatherlabels['Humidity'][i] = 'Uncomfortable - Low'
    # Snow and thunderstorm
    weatherlabels['Snow'][i] = dailyweather['Snow'][i]
    weatherlabels['Thunderstorm'][i] = dailyweather['TStorm'][i]
    # Rain
    if sum(weather[x:o]['Rain']) > 4 or sum(weather[x:o]['PpDur']) > 2:
        weatherlabels.loc[i, 'Rainy'] = 'Yes'
    else:
        weatherlabels.loc[i, 'Rainy'] = 'No'
    # Rain Intensity
    #rainlist = []
    #if weatherlabels.loc[i, 'Rainy'] == 'Yes':
        #for i, number in weather[x:o]['PpLabel'].iteritems():
            #if weather['PpLabel'][i] != 'None':
                #rainlist.append(weather['PpLabel'][i])
                #count = Counter(rainlist)
                #print(count)
                #weatherlabels.loc[i, 'RainIntensity'] == count.most_common()
    #rainlist = []
    
    y = y + 24
    z = z + 24
    x = x + 24
    o = o + 24


weatherlabels.to_csv('weatherlabels.csv')
pd.set_option('display.max_rows', weatherlabels.shape[0]+1)
#print(weatherlabels)
weatherlabels.head(25)

,Day,Temperature,TempVariation,Sunny,Pressure,Humidity,Rainy,Snow,Thunderstorm
0,20130101,Cold,Low,Not Sunny,Low,Uncomfortable - High,Yes,No,No
1,20130102,Cold,Low,Not Sunny,High,Uncomfortable - High,No,No,No
2,20130103,Mild,Low,Not Sunny,High,Uncomfortable - High,No,No,No
3,20130104,Cold,Low,Not Sunny,High,Uncomfortable - High,No,No,No
4,20130105,Cold,Low,Not Sunny,High,Uncomfortable - High,Yes,No,No
5,20130106,Cold,Moderate,Not Sunny,High,Uncomfortable - High,No,No,No
6,20130107,Cold,Low,Not Sunny,High,Uncomfortable - High,Yes,No,No
7,20130108,Cold,Low,Not Sunny,High,Uncomfortable - High,No,No,No
8,20130109,Cold,Moderate,Not Sunny,High,Uncomfortable - High,Yes,No,No
9,20130110,Cold,Moderate,Not Sunny,Low,Uncomfortable - High,Yes,No,No
